In [2]:
import numpy as np
#import pandas as pd
import os

from PIL import Image, ImageEnhance, ImageOps
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


## Function

In [ ]:
base_dir = 'images'  # Remplacez par le chemin de votre répertoire de données
img_width, img_height = 150, 150  # Dimensions des images

In [ ]:
# Fonction pour afficher des images
def plot_images(images, labels, label_encoder, n=9):
    plt.figure(figsize=(10, 10))
    for i in range(n):
        plt.subplot(3, 3, i+1)
        plt.imshow(images[i])
        plt.title(label_encoder.inverse_transform([labels[i]])[0])
        plt.axis('off')
    plt.show()

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(img_width, img_height)):
    
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size)#, Image.LANCZOS)  # Redimensionner l'image
    img_array = np.asarray(img) #   Convertir l'image en tableau numpy
    img_array = img_array / 255.0  # Normalisation
    return img_array

def augment_image(image):
    augmented_images = [image]
    
    # Flip horizontal
    augmented_images.append(np.fliplr(image))
    
    # Rotation
    for angle in [90, 180, 270]:
        augmented_images.append(np.rot90(image, k=angle//90)) 
    
    # Brightness adjustment
    enhancer = ImageEnhance.Brightness(Image.fromarray((image * 255).astype(np.uint8)))
    augmented_images.append(np.asarray(enhancer.enhance(0.5)) / 255.0)  # Darker
    augmented_images.append(np.asarray(enhancer.enhance(1.5)) / 255.0)  # Brighter
    
    # Whitening
    augmented_images.append(np.clip(image + 0.5, 0, 1)) 
    
    return augmented_images

NameError: name 'img_width' is not defined

In [ ]:

# define the base data directory
base_dir = 'images'  # Remplacez par le chemin de votre répertoire de données
img_width, img_height = 150, 150  # Dimensions des images


'''# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(img_width, img_height)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size)#, Image.LANCZOS)  # Redimensionner l'image
    img_array = np.asarray(img) #   Convertir l'image en tableau numpy
    img_array = img_array / 255.0  # Normalisation
    return img_array
'''
# Fonction pour augmenter les images
'''def augment_image(image):
    augmented_images = [image]
    
    # Flip horizontal
    augmented_images.append(np.fliplr(image))
    
    # Rotation
    for angle in [90, 180, 270]:
        augmented_images.append(np.rot90(image, k=angle//90)) 
    
    # Brightness adjustment
    enhancer = ImageEnhance.Brightness(Image.fromarray((image * 255).astype(np.uint8)))
    augmented_images.append(np.asarray(enhancer.enhance(0.5)) / 255.0)  # Darker
    augmented_images.append(np.asarray(enhancer.enhance(1.5)) / 255.0)  # Brighter
    
    # Whitening
    augmented_images.append(np.clip(image + 0.5, 0, 1)) 
    
    return augmented_images'''

# Charger les images et les labels
images = []
labels = []

for label in os.listdir(base_dir):
    label_path = os.path.join(base_dir, label)
    if os.path.isdir(label_path):
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            try:
                img_array = load_and_preprocess_image(image_path)
                augmented_images = augment_image(img_array)
                images.extend(augmented_images)
                labels.extend([label] * len(augmented_images))
            except Exception as e:
                print(f"Erreur de chargement de l'image {image_path}: {e}")
display(images[0].shape)
# Convertir les listes en tableaux numpy
images = np.array(images)
labels = np.array(labels)

# Encoder les labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Fractionner les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)


# Afficher quelques images pour vérification
plot_images(X_train, y_train, label_encoder)
print(images[0].shape)


Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_16718.jpg: name 'load_and_preprocess_image' is not defined
Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_4369.jpg: name 'load_and_preprocess_image' is not defined
Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_16095.jpg: name 'load_and_preprocess_image' is not defined
Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_4157.jpg: name 'load_and_preprocess_image' is not defined
Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_2532.jpg: name 'load_and_preprocess_image' is not defined
Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_4619.jpg: name 'load_and_preprocess_image' is not defined
Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_16491.jpg: name 'load_and_preprocess_image' is not defined
Erreur de chargement de l'image images/n02109961-Eskimo_dog/n02109961_3956.jpg: name 'load_and_prepro

IndexError: list index out of range

In [ ]:


# Définir l'architecture du modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compiler le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=32)


NameError: name 'Sequential' is not defined